# View 3D images using napari

Based on [code](https://github.com/BodenmillerGroup/3D_IMC_publication/blob/main/Python/scripts/single_cell_analysis/LVI_blood_model_analyse_single_cells.ipynb) accompanying the publication by Kuett et al (2022), rewritten and optimized with the help of ChatGPT v4.0.

In [2]:
import os
from tifffile import imread, imwrite
import numpy as np
import napari
from skimage.filters import gaussian
from skimage.segmentation import find_boundaries
from skimage.exposure import rescale_intensity
from matplotlib import colormaps
from matplotlib.colors import to_hex
from napari.utils.colormaps import Colormap, DirectLabelColormap
from napari.utils.colormaps.standardize_color import transform_color
import pandas as pd
import anndata as ad
import copy

In [7]:
## Load mask
file = "../data/Kuett_2022/MainHer2BreastCancerModel/measured_mask_final_segmentation_hwatershed_500.00_90%.tif"
img = imread(file, pattern=None)
img = np.squeeze(img)

In [8]:
napari.view_image(img, scale=[2, 1, 1], ndisplay=3) # FIXME: fading at the end of z-stack

/home/gesa/.py_venv/napari/lib/python3.12/site-packages/napari/plugins/_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)


Viewer(camera=Camera(center=(151.0, 243.5, 325.5), zoom=1.8026639344262294, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(150.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=3, last_used=0, range=((0.0, 304.0, 2.0), (0.0, 488.0, 1.0), (0.0, 652.0, 1.0)), current_step=(75, 243, 325), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'img' at 0x73824774bdd0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x738251a6cae0>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

## View mask by cell type

In [1]:
import numpy as np
import pandas as pd
import tifffile
import napari
from scipy.ndimage import label

In [2]:
# Load the segmentation mask
mask_file = "../data/Kuett_2022/MainHer2BreastCancerModel/measured_mask_final_segmentation_hwatershed_500.00_90%.tif"

segmentation_mask = tifffile.imread(mask_file)
segmentation_mask = segmentation_mask.astype(np.uint32) ## Needs to match cell id type

In [7]:
# Load the phenotypes table
categories_file = '../data/Kuett_2022/MainHer2BreastCancerModel/model201710_cluster_labels_phenograph_recoded.csv'
cell_id_col = 'id' ## column with cell IDs
categories_col = 'ct_broad' ## column with phenotype labels

categories = pd.read_csv(categories_file)
categories = categories.rename(columns={cell_id_col: 'id', categories_col: 'category'})[['id', 'category']] ## standardize columns
categories['id'] = categories['id'].astype(np.uint32) ## Needs to match cell id type
categories['category'].unique()

array(['CD8- T cell', 'myofibroblast', 'other', 'epithelial cell',
       'CD8+ T cell', 'cancer cell', 'B cell', 'stromal cell',
       'endothelial cell', 'macrophage CD44+', 'macrophage cPARP/cCASP3+',
       'macrophage HER2/panCK/Cadherin+', 'plasma cell', 'apoptotic cell'],
      dtype=object)

In [34]:
## Specify phenotypes to display
load_all = False
if load_all:
    cats_to_show = categories['category'].unique()
else:
    cats_to_show = [
        'CD8- T cell',
        'CD8+ T cell',
        'B cell',
        'plasma cell',
        #"cancer cell",
        'macrophage CD44+',
        'macrophage HER2/panCK/Cadherin+',
        'macrophage cPARP/cCASP3+' ## these cells show a cluster
    ]

In [35]:
# Map categories to cell IDs
category_to_cell_ids = {
    category: categories.loc[categories['category'] == category, 'id'].values
    for category in cats_to_show
}

In [36]:
# Prepare Napari viewer
viewer = napari.Viewer(ndisplay=3)

# Function to extract category-specific regions efficiently
def extract_category_mask(segmentation, cell_ids, chunk_size=(50, 100, 100)):
    """
    Extracts a binary mask for the given cell IDs from the segmentation.
    Processes the mask in chunks to save memory.
    """
    binary_mask = np.zeros_like(segmentation, dtype=np.uint8)
    z_size, y_size, x_size = segmentation.shape

    for z_start in range(0, z_size, chunk_size[0]):
        for y_start in range(0, y_size, chunk_size[1]):
            for x_start in range(0, x_size, chunk_size[2]):
                # Define chunk boundaries
                z_end = min(z_start + chunk_size[0], z_size)
                y_end = min(y_start + chunk_size[1], y_size)
                x_end = min(x_start + chunk_size[2], x_size)

                # Extract chunk
                chunk = segmentation[z_start:z_end, y_start:y_end, x_start:x_end]

                # Identify relevant pixels
                chunk_mask = np.isin(chunk, cell_ids)
                binary_mask[z_start:z_end, y_start:y_end, x_start:x_end] = chunk_mask

    return binary_mask

# Process and add each category
for category, cell_ids in category_to_cell_ids.items():
    mask = extract_category_mask(segmentation_mask, cell_ids)
    viewer.add_image(
        mask, 
        name=category,
        scale=[2,1,1],
        colormap='gray', 
        blending='additive', 
        contrast_limits=(0, 1), 
        rendering='attenuated_mip',  # Adjust rendering mode as needed
        opacity=0.5
    )

napari.run()


/home/gesa/.py_venv/napari/lib/python3.12/site-packages/napari/plugins/_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)


## View raw images (FIXME)

In [3]:
## Function to load and preprocess 3D image stack
def load_image_stack(folder_path, crop=None, normalize=True, smooth=True):
    """
    Load and preprocess a 3D stack of images from a folder.

    Args:
        folder_path (str): Path to the folder containing image slices.
        crop (dict): Optional cropping bounds {"x_start", "x_end", "y_start", "y_end"}.
        normalize (bool): Whether to normalize pixel intensities.
        smooth (bool): Whether to apply Gaussian blur.

    Returns:
        np.ndarray: Preprocessed 3D image stack.
    """
    file_paths = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path)
                         if f.endswith(('.tiff', '.tif'))])
    stack = imread(file_paths)
    
    if normalize:
        stack = np.array([rescale_intensity(image, in_range=(image.min(), image.max()), out_range=(0, 65535)) for image in stack], dtype=np.uint16)
    if smooth:
        stack = np.array([gaussian(image, sigma=1, truncate=3, preserve_range=True) for image in stack], dtype=np.uint16)
    if crop:
        stack = stack[:, crop["y_start"]:crop["y_end"], crop["x_start"]:crop["x_end"]]

    return stack

In [4]:
# Function to create a boundary mask
def create_boundaries_mask(label_stack):
    """
    Generate a mask highlighting cell boundaries in a 3D segmentation stack.

    Args:
        label_stack (np.ndarray): 3D stack of labeled segmentation masks.

    Returns:
        np.ndarray: 3D boundary mask.
    """
    return np.array([find_boundaries(slice, connectivity=1, mode='outer', background=0)
                     for slice in label_stack], dtype=label_stack.dtype)

In [5]:
## Function to map phenotypes to cell IDs
def map_clusters_to_labels(label_stack, adata, cluster_column="ct_broad", label_column="id"):
    """
    Efficiently map cluster categories to cell labels in a 3D segmentation stack using vectorized operations.

    Args:
        label_stack (np.ndarray): 3D stack of labeled segmentation masks.
        adata (AnnData): AnnData object with cluster and cell label metadata.
        cluster_column (str): Column name for cluster categories in `adata.obs`.
        label_column (str): Column name for cell labels in `adata.obs`.

    Returns:
        np.ndarray: 3D stack with cluster IDs mapped to cell regions.
    """
    # Create a mapping of cell labels to cluster IDs
    cluster_map = dict(zip(adata.obs[label_column].astype(int), adata.obs[cluster_column].astype("category").cat.codes + 1))
    
    # Initialize an output array with the same shape as the label stack
    mapped_stack = np.zeros_like(label_stack, dtype=np.uint16)
    
    # Vectorized mapping: Replace cell labels with cluster IDs
    unique_labels = np.unique(label_stack)
    for label in unique_labels:
        if label in cluster_map:
            mapped_stack[label_stack == label] = cluster_map[label]

    return mapped_stack

In [6]:
## Function to map phenotypes to colors
def generate_cluster_colors(adata, cluster_column="ct_broad", manual_colors=None, colormap_name="tab20"):
    """
    Generate or retrieve cluster colors for visualization.

    Args:
        adata (AnnData): AnnData object with clustering metadata.
        cluster_column (str): Column name for cluster categories in `adata.obs`.
        manual_colors (list): Optional list of manual colors in hex format.
        colormap_name (str): Name of the matplotlib colormap to use for automatic color generation.

    Returns:
        list: List of hex color codes for clusters.
    """
    unique_clusters = adata.obs[cluster_column].unique()
    num_clusters = len(unique_clusters)
    
    if manual_colors and len(manual_colors) >= num_clusters:
        return manual_colors[:num_clusters]  # Use provided colors if sufficient

    # Generate colors from a colormap
    cmap = colormaps[colormap_name]
    auto_colors = [to_hex(cmap(i / num_clusters)) for i in range(num_clusters)]
    
    return auto_colors


In [7]:
## Function to run napari
def prepare_visualization(image_stacks, cluster_stack, colors, scale_factors):
    """
    Prepare Napari viewer for 3D image visualization in a Jupyter notebook environment.
    
    Args:
        image_stacks (dict): Dictionary of named 3D image stacks.
        cluster_stack (np.ndarray): 3D stack with cluster labels.
        colors (list): List of hex color codes for clusters.
        scale_factors (list): Scaling factors for Napari.
    """
    # Create the viewer
    viewer = napari.Viewer(ndisplay=3)

    # Add image channels to the viewer
    for name, stack in image_stacks.items():
        viewer.add_image(stack, name=name, scale=scale_factors)

    # Add the cluster labels as a labels layer using the DirectLabelColormap
    cluster_colors = [CONFIG["napari_color_background"]] + colors
    cmap = DirectLabelColormap(colors=cluster_colors)  # Create the colormap
    viewer.add_labels(cluster_stack, name="Clusters", colormap=cmap, scale=scale_factors)

    return viewer  # Return the viewer object for interaction


In [8]:
## Load marker intensities & phenotypes
data_dir = "../data/Kuett_2022/MainHer2BreastCancerModel/" ## Path to csv files with cluster labels and mean intensities.
phenotypes_file = data_dir + 'model201710_cluster_labels_phenograph_recoded.csv'
intensities_file = data_dir + 'model201710_mean_intensities.csv'

phenotype_category = 'ct_broad' ## Which cluster labels to use: 'phenograph' or 'ct_broad'.

phenotypes = pd.read_csv(phenotypes_file)
intensities = pd.read_csv(intensities_file)
adata = ad.AnnData(X=intensities.values,
                   obs=phenotypes[['id', phenotype_category]])

/home/gesa/.py_venv/napari/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [12]:
label_stack.shape

(152, 488, 652)

In [9]:
## Load segmentation labels
label_stack_path = "../data/Kuett_2022/MainHer2BreastCancerModel/measured_mask_final_segmentation_hwatershed_500.00_90%.tif" ## Path to the 3D segmentation label stack.

label_stack = imread(label_stack_path)
boundaries_mask = create_boundaries_mask(label_stack)
internal_labels = np.multiply(np.logical_not(boundaries_mask), label_stack)

In [10]:
## Map cluster labels to segmentation
cluster_stack = map_clusters_to_labels(internal_labels, adata, cluster_column=phenotype_category)

KeyboardInterrupt: 

In [26]:
# Save data (temporary)
outfile = "../data/temp/Kuett_2022_cluster_stack.tif"
imwrite(outfile, cluster_stack)

In [ ]:
## Load data (temporary)
infile = "../data/temp/Kuett_2022_cluster_stack.tif"
cluster_stack = imread(infile)

In [ ]:
## Load channel information for channel selection
panel_path = "../data/Kuett_2022/MainHer2BreastCancerModel/model201710_panel.csv"

panel = pd.read_csv(panel_path)
panel = panel.loc[ panel['MeasureMask']==1 ] ## Only show channels included in the final image stack
channel_mapping = dict(zip(panel['clean_target'], panel['Metal Tag']))
panel

,Metal Tag,Target,clean_target,Antibody Clone,Final Concentration / Dilution,singleTIFFs,MeasureMask
8,In113,Histone H3,Histone H3,D1H2,2 ug/mL,1,1
11,Pr141,Cytokeratin 5,CK5,EP1601Y,2 ug/mL,1,1
12,Nd143,Cytokeratin 19,CK19,Troma-III,6 ug/mL,1,1
13,Nd144,Cytokeratin 8/18,CK8/18,C51,4 ug/mL,1,1
15,Nd146,CD68,CD68,KP1,3 ug/mL,1,1
16,Sm147,Keratin 14 (KRT14),CK14,polyclonal_PA5-16722 USE CLONE LL002,NaN,1,1
17,Nd148,SMA,SMA,1A4,2 ug/mL,1,1
18,Sm149,Vimentin,Vimentin,D21H3,2 ug/mL,1,1
19,Nd150,CD138 (Syndecan-1),CD138,MI15,8 ug/mL,1,1
21,Sm152,CD3 epsilon,CD3,D7A6E,2 ug/mL,1,1


In [ ]:
## Configuration
CONFIG = {
    "scale_factors": [2, 1, 1],  # Scaling for Napari visualization
    "crop": None,  # Example: {"x_start": 100, "x_end": 500, "y_start": 50, "y_end": 450}
    "channels": [ ## Channels to display (from panel['clean_target'])
        #"Ir193",
        #"HER2 (bis)",
        "CD68",
        "cPARP+cCasp3",
    ],
    "cluster_colors": generate_cluster_colors(adata, cluster_column="ct_broad", manual_colors=None),
    "napari_color_background": '#000000',  # Background color in Napari
}

NameError: name 'generate_cluster_colors' is not defined

In [ ]:
# Load image stacks
base_folder = "../data/Kuett_2022/MainHer2BreastCancerModel/SIMILARITY10_Nd148" ## Base folder containing channel folders.

image_stacks = {channel: load_image_stack(os.path.join(base_folder, channel_mapping[channel]))
                for channel in CONFIG["channels"]}

In [ ]:
# Prepare visualization
prepare_visualization(image_stacks, cluster_stack, CONFIG["cluster_colors"], CONFIG["scale_factors"])

Viewer(camera=Camera(center=(151.0, 827.0, 801.5), zoom=0.5625377643504531, angles=(0.0, 0.0, 89.99999999999999), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=10, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=3, last_used=0, range=((0.0, 304.0, 2.0), (0.0, 1655.0, 1.0), (0.0, 1604.0, 1.0)), current_step=(75, 827, 801), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Ir193' at 0x741d64b7bc20>, <Image layer 'HER2 (bis)' at 0x741d64b55850>, <Image layer 'CD68' at 0x741d05f0acf0>, <Image layer 'cPARP+cCasp3' at 0x741d05fcb530>, <Labels layer 'Clusters' at 0x741cffdaa2a0>], help='use <1> for activate the label eraser, use <2> for activate the paint brush, use <3> for activate the fill bucket, use <4> for pick mode', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, 

: 

In [ ]:
# Importing required libraries
from tifffile import imread, imsave  # For reading and saving TIFF images
import os, sys, cv2  # OS for file operations, sys for system-level path management, and cv2 for image processing
import numpy as np  # Numerical operations
import scanpy as sc  # Single-cell analysis toolkit
import pandas as pd  # Data manipulation
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Enhanced plotting
from matplotlib import rcParams  # Configuration of matplotlib settings
import napari  # Image visualization in 2D/3D
import copy  # For deep copying objects
from skimage.segmentation import find_boundaries  # To detect boundaries in 2D images

# Importing colormap utilities from napari for custom visualizations
from napari.utils.colormaps.colormaps import Colormap
from napari.utils.colormaps.standardize_color import transform_color

# Loading the AnnData object containing cell metadata and clustering information
adata = sc.read_h5ad(results_file)  # This file contains columns such as cell_labels, cell_volume, phenograph

# Helper function to create file paths for a 3D stack of images
def image_filepath_for_3D_stack(img_folder, tiff=True):
    """
    Generates a list of file paths for all TIFF files in the specified folder.
    Ensures filenames are sorted for proper stacking.
    """
    img_list = []
    channel_files = sorted(os.listdir(img_folder))
    for file in channel_files:
        if tiff and (file.endswith('.tiff') or file.endswith('.tif')):
            img_list.append(os.path.join(img_folder, file))
        elif not tiff:
            img_list.append(os.path.join(img_folder, file))
    return img_list

# Function to load and preprocess 3D image stacks
def load_channel_stack_for_napari(channel_name_to_load, base_folder, missing, crop_im=True):
    """
    Loads a 3D image stack, interpolates missing slices, and applies Gaussian blur and normalization.
    Optionally crops the image to the specified dimensions.
    """
    metal_folder = os.path.join(base_folder, channel_name_to_load)
    image_path1 = image_filepath_for_3D_stack(metal_folder)
    image1 = imread(image_path1)
    
    # Handling missing slices by interpolation
    if missing is not None:
        missing_slice_image = np.mean(np.array([image1[missing-1], image1[missing+1]]), axis=0)
        image1 = np.insert(image1, missing, missing_slice_image, axis=0)
    
    # Preprocessing each slice: normalization, clipping, and Gaussian blur
    for i in range(image1.shape[0]):
        tmp_im = cv2.normalize(image1[i], None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        tmp_im = np.clip(tmp_im, 0, 65535)
        image1[i] = cv2.GaussianBlur(tmp_im, (3, 3), 1)
    
    # Optional cropping
    if crop_im:
        image1 = image1[:, y_start:y_end, x_start:x_end]
    
    print('Max pixel value:', np.max(image1))
    print('Median pixel value:', np.percentile(image1, 50))
    return image1

# Creating boundary masks for each slice in the 3D stack
boundaries_only = np.zeros(cell_labels.shape, dtype=cell_labels.dtype)
for k in range(boundaries_only.shape[0]):
    slice_2D = cell_labels[k]
    boundaries_only[k] = find_boundaries(slice_2D, connectivity=1, mode='outer', background=0)

# Mask to retain only cell interiors by removing boundaries
with_boundaries_mask = np.multiply(np.logical_not(boundaries_only), cell_labels)

# Mapping cluster labels to the segmentation image
cluster_labels = list(adata.obs['phenograph'])  # Clustering categories
object_labels = list(adata.obs['cell_labels'])  # Cell IDs

cluster_labels_image = copy.deepcopy(with_boundaries_mask)
for item in range(len(object_labels)):
    obi = int(object_labels[item])
    cluster_labels_image[with_boundaries_mask == obi] = int(cluster_labels[item])

# Save the labeled image
cluster_labels_image = cluster_labels_image.astype('uint16')
imsave(cluster_labels_name, cluster_labels_image)

# Reloading the saved labeled image
cluster_labels_image = imread(cluster_labels_name)
cluster_labels_image = np.squeeze(cluster_labels_image)

# Define color maps for clusters
cluster_colors = ['#1f78b4', '#b2df8a', '#33a02c', ... ]  # List of hex color codes
cluster_colors_napari = ['#000000'] + cluster_colors  # Adding black for background
cmap = Colormap(transform_color(cluster_colors_napari))  # Create a colormap

# Create a dictionary mapping cluster IDs to colors for Napari
napari_color_dict = {i: cluster_colors_napari[i] for i in range(len(cluster_colors_napari))}
colors = {label: transform_color(color_str)[0] for label, color_str in napari_color_dict.items()}

# Define scale factors for visualization
scale_factors = [2, 1, 1]

# Load different channels into 3D stacks
stack1 = load_channel_stack_for_napari('Ir191', stack_registred, None, False)
stack2 = load_channel_stack_for_napari('Pr141', stack_registred, None, False)
stack3 = load_channel_stack_for_napari('Dy164', stack_registred, None, False)
stack4 = load_channel_stack_for_napari('Yb171', stack_registred, None, False)

# Visualize the data in Napari
with napari.gui_qt():
    viewer = napari.view_image(stack1[:, row_start:row_end, col_start:col_end], name='Ir191', scale=scale_factors)
    viewer.add_image(stack2[:, row_start:row_end, col_start:col_end], name='GLUT1', scale=scale_factors)
    viewer.add_image(stack3[:, row_start:row_end, col_start:col_end], name='CD20', scale=scale_factors)
    viewer.add_image(stack4[:, row_start:row_end, col_start:col_end], name='CD4', scale=scale_factors)
    viewer.add_labels(cluster_labels_image, name='clusters', color=colors, scale=scale_factors)
